In [2]:
%load_ext autoreload
%autoreload 2
from main  import *
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"

In [3]:
opt = Config()
opt = Config()    
opt.caption_data_path = 'caption.pth' # 原始数据
opt.test_img = 'img/example.jpeg' # 输入图片
opt.use_gpu = False  # 是否使用GPU(没必要)
#opt.model_ckpt='caption_0914_1947' # 预训练的模型
opt.img_feature_path = 'results.pth'

# 数据
vis = Visualizer(env = opt.env)
dataloader = get_dataloader(opt)
_data = dataloader.dataset._data
word2ix,ix2word = _data['word2ix'],_data['ix2word']

# 模型
model = CaptionModel(opt,word2ix,ix2word)
if opt.model_ckpt:
    model.load(opt.model_ckpt)
optimizer = model.get_optimizer(opt.lr1,opt.lr2)
criterion = t.nn.CrossEntropyLoss()
if opt.use_gpu:
    model.cuda()
    criterion.cuda()

for epoch in range(opt.epoch):        
    loss_meter.reset()
    for ii,(imgs, (captions, lengths),indexes)  in tqdm.tqdm(enumerate(dataloader)):
        # 训练
        optimizer.zero_grad()
        input_captions = captions[:-1]
        if opt.use_gpu:
            imgs = imgs.cuda()
            captions = captions.cuda()
        imgs = Variable(imgs)
        captions = Variable(captions)
        input_captions = captions[:-1]
        target_captions = pack_padded_sequence(captions,lengths)[0]
        score,_ = model(imgs,input_captions,lengths)
        loss = criterion(score,target_captions)
        loss.backward()
        optimizer.step()
        loss_meter.add(loss.data[0])

        # 可视化
        if (ii+1)%opt.plot_every ==0:
            if os.path.exists(opt.debug_file):
                ipdb.set_trace()

            vis.plot('loss',loss_meter.value()[0])

            # 可视化原始图片 + 可视化人工的描述语句
            raw_img = _data['ix2id'][indexes[0]]
            img_path=opt.img_path+raw_img
            raw_img = Image.open(img_path).convert('RGB')
            raw_img = tv.transforms.ToTensor()(raw_img)

            raw_caption = captions.data[:,0]
            raw_caption = ''.join([_data['ix2word'][ii] for ii in raw_caption])
            vis.text(raw_caption,u'raw_caption')
            vis.img('raw',raw_img,caption=raw_caption)

            # 可视化网络生成的描述语句
            results = model.generate(imgs.data[0])
            vis.text('</br>'.join(results),u'caption')
    model.save()

FileNotFoundError: [Errno 2] No such file or directory: 'results.pth'

In [25]:
opt.use_gpu

False

In [5]:
optimizer = model.get_optimizer(opt.lr1)
criterion = t.nn.CrossEntropyLoss()
if opt.use_gpu:
    model.cuda()
    criterion.cuda()

# 统计
loss_meter = meter.AverageValueMeter()


In [23]:
opt.img_path='/home/cy/caption_data/'

151it [01:45,  1.31it/s]Process Process-24:
Process Process-22:
Process Process-21:
Process Process-23:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in 

KeyboardInterrupt: 